In [30]:
%run /home/jovyan/work/database_operations/spark_db_connection.ipynb import create_spark_session
%run /home/jovyan/work/model/src/model_deployment.ipynb import Classifier, ModelTraining
%run /home/jovyan/work/database_operations/db_operations.ipynb import DataOperations
%run /home/jovyan/work/model/src/transform_data.ipynb import TransformData
%run /home/jovyan/work/model/process/preprocess.ipynb import preprocess
%run /home/jovyan/work/model/process/visualize.ipynb import visualize
%run /home/jovyan/work/model/process/evaluate.ipynb import evaluate
%run /home/jovyan/work/model/process/ETL.ipynb import ETL_pipeline

In [31]:
import logging

In [32]:
def classification_pipeline(epochs: int, 
                            data_path: str = "/home/jovyan/work/dataset/results.csv", 
                            table_name: str = "results") -> None:
    """
    Executes a full pipeline for training and evaluating a multi-input, multi-output, 
    multi-class neural network model for classification tasks.

    This pipeline performs the following steps:
    1. **Data Extraction**: Connects to a PostgreSQL database and saves raw data.
    2. **Data Transformation**: Transforms and prepares the data for model training, including defining schemas.
    3. **Data Loading**: Loads the data to the database.
    4. **Model Preparation**: Constructs a TensorFlow neural network model with multiple inputs and outputs.
    5. **Model Training**: Trains the model using the specified number of epochs.
    6. **Model Evaluation**: Evaluates the trained model on test data and computes performance metrics.
    7. **Visualization**: Visualizes model structure, performance and prediction results.

    Args:
        epochs: Number of epochs for training neural network
        data_path: The file path to the CSV file connecting raw data to extract
        table_name: Name of the table in PostgreSQL databse, where the data is stored
    """
    try:
        logging.info("Started classification pipeline \n")
        spark = create_spark_session()
        etl_pipeline(spark=spark, data_path=data_path, table_name=table_name)
        
        data_oper = DataOperations(spark)
        df = data_oper.load_data(table_name="cleaned_data")

        transform_oper = TransformData()
        categorical_features, numeric_features, targets = transform_oper.describe_features_types()
        number_categories = transform_oper.number_of_categories(df, targets)

        training_dataset, test_dataset, validation_dataset, merge_models, inputs = preprocess(spark=spark,
                                                                                              categorical_features=categorical_features,
                                                                                              numeric_features=numeric_features,
                                                                                              targets=targets)
        model = Classifier()
        model_deployed, metrics = model.model_deploy(merge_models=merge_models, 
                                                     inputs=inputs, 
                                                     n_classes=number_categories)
        model_training = ModelTraining()
        trained_model, history = model_training.model_train(model=model_deployed, 
                                                            training_dataset=training_dataset, 
                                                            validation_dataset=validation_dataset,
                                                            epochs=epochs)

        home_conf_matrix, away_conf_matrix = evaluate(model=trained_model,
                                                      test_data=test_dataset,
                                                      metrics=metrics,
                                                      model_type="classifier")

        visualize(model_deployed, home_conf_matrix, away_conf_matrix, history, metrics)

        logging.info("Finished classification pipeline")
    except Exception as e:
        logging.error(f"Error while executing classification pipeline: {e}")
        raise e




In [33]:
classification_pipeline(3)

Started classification pipeline 

Started ETL process
Successfully ingested data from: /home/jovyan/work/dataset/results.csv
Successfully saved table: results
Successfully loaded table: results

Successfully deleted emty fields
Successfully changed date into years
Successfully filtered data
Successfully converted string features into numeric
Successfully divided data into training, validation and test datasets
Successfully standardized datasets
Successfully saved table: cleaned_data
Successfully saved table: train
Successfully saved table: val
Successfully saved table: test
Successfully finished ETL process 

Successfully loaded table: cleaned_data
Started preprocessing
Successfully loaded table: cleaned_data
Successfully loaded table: train
Successfully loaded table: val
Successfully loaded table: test


INFO:tensorflow:Enabled check-numerics callback in thread MainThread


Enabled check-numerics callback in thread MainThread
Successfully created inputs and embedding layers for model deployment
Successfully prepared dataset for model: training
Successfully prepared dataset for model: validation
Successfully prepared dataset for model: test
Successfully finished preprocessing 



Model: "functional_7"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_away_team     │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_city          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_country       │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_home_team     │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_year          │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ input_tournament    │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_42        │ (None, 1, 50)     │     16,450 │ input_away_team[… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_43        │ (None, 1, 50)     │     98,250 │ input_city[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_44        │ (None, 1, 50)     │     12,250 │ input_country[0]… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_45        │ (None, 1, 50)     │     16,450 │ input_home_team[… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_46        │ (None, 1, 23)     │      1,058 │ input_year[0][0]  │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ embedding_47        │ (None, 1, 50)     │      7,200 │ input_tournament… │
│ (Embedding)         │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_42          │ (None, 50)        │          0 │ embedding_42[0][… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_43          │ (None, 50)        │          0 │ embedding_43[0][… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_44          │ (None, 50)        │          0 │ embedding_44[0][… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_45          │ (None, 50)        │          0 │ embedding_45[0][… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ reshape_46          │ (None, 23)        │          0 │ embedding_46[0][

...

Successfully trained the model

Started model evaluation
Data prediction:


220/220 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step


Successfully predicted data on the model

Calculating metrics:


220/220 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - away_score_loss: 1.7531 - away_score_sparse_categorical_accuracy: 0.3008 - home_score_loss: 1.4623 - home_score_sparse_categorical_accuracy: 0.2782 - loss: 3.6919



Loss (sparse_categorical_crossentropy): 3.686765670776367 
'home_score' loss (sparse_categorical_crossentropy): 1.4566271305084229 
'away_score' loss (sparse_categorical_crossentropy): 1.7532421350479126 
'home_score' sparse_categorical_accuracy: 0.30245155096054077 
'away_score' sparse_categorical_accuracy: 0.28463512659072876
Successfully finished model evaluation 

Started viusalization process
Successfully saved model visualization into file: /home/jovyan/work/model/plots/model_schema.png
Successfully saved an image to the file: /home/jovyan/work/model/plots/home_score_conf_mat
Successfully saved an image to the file: /home/jovyan/work/model/plots/away_score_conf_mat
Successfully saved an image to the file: /home/jovyan/work/model/plots/sparse_categorical_crossentropy
Successfully saved an image to the file: /home/jovyan/work/model/plots/sparse_categorical_accuracy
Successfully finished visualization process 

Finished classification pipeline
